# Create Word Embedding for Graph

In [2]:
from konlpy.tag import Okt
Tokenizer = Okt()

import os
import json

path = os.path.dirname(os.getcwd())
print(path)

/home/vips/share/Vu


In [3]:
# pos_remove = ["Eomi", "Punctuation", "Unknown", "KoreanParticle", "Hashtag", "ScreenName",
#                     "Email", "URL", "Josa", "PreEomi", "Foreign"]
pos_remove = ["Eomi", "Punctuation", "Unknown", "KoreanParticle", "Hashtag", "ScreenName",
                    "Email", "URL", "PreEomi", "Foreign"]

def simple_process_tokens(sent):
    tokens = Tokenizer.pos(sent, stem=True, norm=True)
    out = []
    for (token, type) in tokens:
        if(type in pos_remove): 
            continue
        else:
            out.append(token)
    return out

## Read data

In [5]:
texts_ko = {}

In [6]:
sentences = []

with open(os.path.join("NLP_data", "kornli", "multinli.train.ko.tsv"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += line.split("\t")[:-1]

with open(os.path.join("NLP_data", "kornli", "xnli.dev.ko.tsv"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += line.split("\t")[:-1]

with open(os.path.join("NLP_data", "kornli", "xnli.test.ko.tsv"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += line.split("\t")[:-1]

print(len(sentences))

texts_ko.update({"kornli": [simple_process_tokens(doc) for doc in sentences]})

800404


In [7]:
sentences = []

with open(os.path.join("NLP_data", "korsts", "sts-dev.tsv"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += line.split("\t")[-2:]

with open(os.path.join("NLP_data", "korsts", "sts-test.tsv"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += line.split("\t")[-2:]

with open(os.path.join("NLP_data", "korsts", "sts-train.tsv"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += line.split("\t")[-2:]

print(len(sentences))

texts_ko.update({"korsts": [simple_process_tokens(doc) for doc in sentences]})

17256


In [8]:
sentences = []

with open(os.path.join("NLP_data", "nsmc", "ratings_train.txt"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += [line.split("\t")[1]]

with open(os.path.join("NLP_data", "nsmc", "ratings_test.txt"), 'r') as f:
    for line in f.readlines()[1:]:
        sentences += [line.split("\t")[1]]

print(len(sentences))

# texts_ko += [process_tokens(doc) for doc in sentences]
texts_ko.update({"nsmc": [simple_process_tokens(doc) for doc in sentences]})

200000


In [9]:
sentences = []

with open(os.path.join("NLP_data", "naver-ner", "train.tsv"), 'r') as f:
    for line in f.readlines():
        sentences += [line.split("\t")[0]]

with open(os.path.join("NLP_data", "naver-ner", "test.tsv"), 'r') as f:
    for line in f.readlines():
        sentences += [line.split("\t")[0]]

print(len(sentences))


texts_ko.update({"ner": [simple_process_tokens(doc) for doc in sentences]})

90000


In [10]:
print(len(texts_ko))
with open('NLP_data/Tokens_dict.json', 'w', encoding= 'utf-8') as fp:
    json.dump(texts_ko, fp,ensure_ascii=False)

4


### remove stop_words

In [3]:
with open(os.path.join("NLP_data", "stop_words.json"), 'r') as f:
    stop_words = json.load(f)

def stop_words_remove(sent):
    new_tokens = []
    for token in sent:
        if(token in stop_words):
            continue
        new_tokens.append(token)
    return new_tokens

In [4]:
Tokens_dict = {
    "nsmc": [],
    "kornli": [],
    "korsts": [],
    "ner": []
}

with open(os.path.join("NLP_data", "Tokens_dict.json"), 'r') as f:
    data = json.load(f)
    
    for key in Tokens_dict:
        print(key)
        for sent in data[key]:
            Tokens_dict[key].append(stop_words_remove(sent))

with open('NLP_data/Tokens_dict_remove_stop_words.json', 'w', encoding= 'utf-8') as fp:
    json.dump(Tokens_dict, fp,ensure_ascii=False)

nsmc
kornli
korsts
ner


In [5]:
with open(os.path.join("NLP_data", "Tokens_dict_remove_stop_words.json"), 'r') as f:
    data = json.load(f)
    nscm_data = data["nsmc"]


sentences = []
i = 0
with open(os.path.join("NLP_data", "nsmc", "ratings_train.txt"), 'r') as f:
    for line in f.readlines()[1:]:
        sent = line.split("\t")
        sentences += [(sent[1],  nscm_data[i], int(sent[2]))]
        i += 1

with open(os.path.join("NLP_data", "nsmc_train.json"), 'w') as fp:
    json.dump(sentences, fp,ensure_ascii=False)

sentences = []

with open(os.path.join("NLP_data", "nsmc", "ratings_test.txt"), 'r') as f:
    for line in f.readlines()[1:]:
        sent = line.split("\t")
        sentences += [(sent[1],  nscm_data[i], int(sent[2]))]
        i += 1

with open(os.path.join("NLP_data", "nsmc_test.json"), 'w') as fp:
    json.dump(sentences, fp,ensure_ascii=False)

## Training word2vec

In [6]:
concat_texts_ko = Tokens_dict["kornli"] + Tokens_dict["korsts"] + Tokens_dict["ner"] + Tokens_dict["nsmc"]
print(len(concat_texts_ko))

1107660


In [ ]:
from gensim.models import Word2Vec


model = Word2Vec(sentences=concat_texts_ko, vector_size=128, window=5, min_count=5, workers=4)
model.save("ko_word2vec.model")